# Tutorial Part 3,4

#### ***[코드출처: 오늘코드(박조은 님)](https://github.com/corazzon)***

![](https://static.amazon.jobs/teams/53/images/IMDb_Header_Page.jpg?1501027252)

## 첫 번째 시도 (average feature vectors)
- 튜토리얼2의 코드로 벡터의 평균을 구한다.

## 두 번째 시도 (K-means)
- Word2Vec은 의미가 관련있는 단어들의 클러스터를 생성하기 때문에 클러스터 내의 단어 유사성을 이용하는 것이다.
- 이런식으로 벡터를 그룹화 하는 것을 'Vector Quantization'(벡터 양자화)라고 한다.
- 이를 위해서는 K-means와 같은 클러스터링 알고리즘을 사용하여 클러스터라는 단어의 중심을 찾아야 한다.
- 비지도 학습인 K-means를 통해 클러스터링을 하고 지도학습인 랜덤포레스트로 리뷰가 추천인지 아닌지를 예측한다.

In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier

from bs4 import BeautifulSoup
import re
import time

from nltk.corpus import stopwords
import nltk.data
from nltk.stem.snowball import SnowballStemmer
from multiprocessing import Pool

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
df_train = pd.read_csv("../input/nlp-dataset/labeledTrainData.tsv", header = 0,
                      delimiter = "\t", quoting = 3)
df_test = pd.read_csv("../input/nlp-dataset/testData.tsv", header = 0,
                      delimiter = "\t", quoting = 3)
df_unlabeled = pd.read_csv("../input/nlp-dataset/unlabeledTrainData.tsv", header = 0,
                      delimiter = "\t", quoting = 3)

In [3]:
class KaggleWord2VecUtility(object):

    @staticmethod
    def review_to_wordlist(review, remove_stopwords=False):
        # 1. HTML 제거
        review_text = BeautifulSoup(review, "html.parser").get_text()
        # 2. 특수문자를 공백으로 바꿔줌
        review_text = re.sub('[^a-zA-Z]', ' ', review_text)
        # 3. 소문자로 변환 후 나눈다.
        words = review_text.lower().split()
        # 4. 불용어 제거
        if remove_stopwords:
            stops = set(stopwords.words('english'))
            words = [w for w in words if not w in stops]
        # 5. 어간추출
        stemmer = SnowballStemmer('english')
        words = [stemmer.stem(w) for w in words]
        # 6. 리스트 형태로 반환
        return(words)

    @staticmethod
    def review_to_join_words( review, remove_stopwords=False ):
        words = KaggleWord2VecUtility.review_to_wordlist(\
            review, remove_stopwords=False)
        join_words = ' '.join(words)
        return join_words

    @staticmethod
    def review_to_sentences( review, remove_stopwords=False ):
        # punkt tokenizer를 로드한다.
        """
        이 때, pickle을 사용하는데
        pickle을 통해 값을 저장하면 원래 변수에 연결 된 참조값 역시 저장된다.
        저장된 pickle을 다시 읽으면 변수에 연결되었던
        모든 레퍼런스가 계속 참조 상태를 유지한다.
        """
        tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        # 1. nltk tokenizer를 사용해서 단어로 토큰화 하고 공백 등을 제거한다.
        raw_sentences = tokenizer.tokenize(review.strip())
        # 2. 각 문장을 순회한다.
        sentences = []
        for raw_sentence in raw_sentences:
            # 비어있다면 skip
            if len(raw_sentence) > 0:
                # 태그제거, 알파벳문자가 아닌 것은 공백으로 치환, 불용어제거
                sentences.append(\
                    KaggleWord2VecUtility.review_to_wordlist(\
                    raw_sentence, remove_stopwords))
        return sentences


    # 참고 : https://gist.github.com/yong27/7869662
    # http://www.racketracer.com/2016/07/06/pandas-in-parallel/
    # 속도 개선을 위해 멀티 스레드로 작업하도록
    @staticmethod
    def _apply_df(args):
        df, func, kwargs = args
        return df.apply(func, **kwargs)

    @staticmethod
    def apply_by_multiprocessing(df, func, **kwargs):
        # 키워드 항목 중 workers 파라메터를 꺼냄
        workers = kwargs.pop('workers')
        # 위에서 가져온 workers 수로 프로세스 풀을 정의
        pool = Pool(processes=workers)
        # 실행할 함수와 데이터프레임을 워커의 수 만큼 나눠 작업
        result = pool.map(KaggleWord2VecUtility._apply_df, [(d, func, kwargs)
                for d in np.array_split(df, workers)])
        pool.close()
        # 작업 결과를 합쳐서 반환
        return pd.concat(result)
    
    
# KaggleWord2VecUtility를 class로 생성하여 사용 
# 코드 출처: https://github.com/corazzon/KaggleStruggle/blob/master/word2vec-nlp-tutorial/KaggleWord2VecUtility.py

In [4]:
sentences = []
for review in df_train["review"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(
    review, remove_stopwords = False)
    
# KaggleWord2VecUtility을 사용하여 train 데이터를 정제해준다.

/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:314: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:389: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [5]:
for review in df_unlabeled["review"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(
    review, remove_stopwords = False)

/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:389: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:389: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:389: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behin

In [6]:
num_features = 300 # 문자 벡터 차원 수 (size)
min_word_count = 40 # 최소 문자 수 (min_count)
num_workers = 4 # 병렬 처리 스레드 수 (workers)
context = 10 # 문자열 창 크기 (window)
downsampling = 1e-3 # 문자 빈도 수 Downsample (sample)

model = Word2Vec(sentences, workers = num_workers,
                 size = num_features, min_count = min_word_count,
                 window = context, sample = downsampling)

model

In [7]:
# 숫자로 단어를 표현
# Word2Vec 모델은 어휘의 각 단어에 대한 feature 벡터로 구성되며
# 'syn0'이라는 넘파이 배열로 저장된다.
# syn0의 행 수는 모델 어휘의 단어 수
# 컬럼 수는 part 2에서 설정한 피처 벡터의 크기
type(model.wv.syn0)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


numpy.ndarray

In [8]:
model.wv.syn0.shape

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


(11986, 300)

In [9]:
model.wv["flower"].shape

(300,)

In [10]:
model.wv["flower"][:10]

array([-1.028998  ,  0.8449268 ,  0.061165  ,  0.13225688,  0.06006935,
       -0.33765045, -0.1157001 , -0.6477264 ,  0.26683876,  0.564404  ],
      dtype=float32)

# K-means 클러스터링으로 데이터 묶기
- [k-평균 알고리즘 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/K-%ED%8F%89%EA%B7%A0_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98)
- 클러스터링은 비지도 학습 기법
- 클러스터링은 유사성 등 개념에 기초해 몇몇 그룹으로 분류하는 기법
- 클러스터링의 목적은 샘플(실수로 구성된 n차원의 벡터)을 내부적으로는 비슷하지만 외부적으로 공통 분모가 없는 여러 그룹으로 묶는 것 
- 특정 차원의 범위가 다른 차원과 차이가 크면 클러스터링 하기 전에 스케일을 조정해야 한다.

   1. 최초 센트로이드(centroid)(중심점)로 K개의 벡터를 무작위로 선정한다.
   2. 각 샘플을 그 위치에서 가장 가까운 센트로이드에 할당한다.
   3. 센트로이드의 위치를 재계산한다.
   4. 센트로이드가 더 이상 움직이지 않을 때까지 2와 3을 반복한다.
   
참고: [책]모두의 데이터 과학(with 파이썬)

In [11]:
# 단어 벡터에서 k-means를 실행하고 일부 클러스터를 찍어본다.
start = time.time()

# 클러스터의 크기 "k"를 어휘 크기의 1/5 이나 평균 5단어로 설정한다.
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] / 5
num_clusters = int(num_clusters)

# K-means를 정의하고  학습시킨다.
kmeans_clustering = KMeans(n_clusters = num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

# 끝난시간에서 시작시간을 빼서 걸린 시간을 구한다.
end = time.time()
elapsed = end - start

print("Time taken for K Means clustering: ", elapsed, "seconds.")

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """


Time taken for K Means clustering:  892.3186316490173 seconds.


In [12]:
# 각 어휘 단어를 클러스터 번호에 매핑되게 word/index 사전을 만든다.
idx = list(idx)
names = model.wv.index2word
word_centroid_map = {names[i]: idx[i] for i in range(len(names))}
# word_cenetroid_map = dict(zip(model.wv.index2word, idx))

# 첫번째 클러스터의 처음 10개를 출력
for cluster in range(0, 10):
    # 클러스터 번호를 출력
    print("\n Cluster {}".format(cluster))
    
    # 클러스터 번호와 클러스터에 있는 단어를 찍는다.
    words = []
    for i in range(0, len(list(word_centroid_map.values()))):
        if(list(word_centroid_map.values())[i] == cluster):
            words.append(list(word_centroid_map.keys())[i])
    print(words)


 Cluster 0
['burton']

 Cluster 1
['lean', 'pen', 'scriptwrit', 'stroke', 'bravo', 'congratul', 'polanski', 'historian', 'poirot', 'biopic', 'bunuel', 'bakshi', 'crumb', 'bruno', 'rewrit', 'scarfac', 'rever', 'cameraman', 'filmographi', 'modesti', 'mileston', 'ki', 'ritchi', 'fincher', 'signatur', 'essay', 'hartley', 'consult', 'kazan', 'prolif', 'vern', 'truman', 'renown', 'chabrol', 'weir', 'autobiographi', 'foss', 'biograph', 'lovecraft', 'besson', 'godard', 'scorces', 'peckinpah', 'antonioni', 'hugo', 'gilliam', 'boorman', 'memoir', 'clanci', 'ozon', 'araki', 'solondz', 'werner', 'herzog', 'adamson', 'kitano', 'yuzna', 'alexandr', 'abel', 'visconti', 'tolkien', 'autobiograph', 'capra', 'fleischer', 'cypher', 'somerset', 'tod', 'corbucci', 'hark', 'fassbind', 'maltin', 'mamet', 'attenborough', 'jarmusch', 'bruckheim', 'novic', 'mulholland', 'excalibur', 'ferrara', 'varga', 'vertigo', 'truffaut', 'elia', 'mckenzi', 'maugham', 'angelopoulo', 'capot', 'visionari', 'caruso', 'raoul', '

In [13]:
"""
판다스로 데이터 프레임 형태의 데이터로 읽어온다.

그리고 이전 튜토리얼에서 했던 것 처럼 clean_train_review와
clean_test_review로 텍스트를 정제한다.
"""

'\n판다스로 데이터 프레임 형태의 데이터로 읽어온다.\n\n그리고 이전 튜토리얼에서 했던 것 처럼 clean_train_review와\nclean_test_review로 텍스트를 정제한다.\n'

In [14]:
clean_train_reviews = []
for review in df_train["review"]:
    clean_train_reviews.append(
        KaggleWord2VecUtility.review_to_wordlist(review,
                                                remove_stopwords = True))

In [15]:
clean_test_reviews = []
for review in df_test["review"]:
    clean_test_reviews.append(
        KaggleWord2VecUtility.review_to_wordlist(review,
                                                remove_stopwords = True))

In [16]:
# bag of centroids 생성
# 속도를 위해 centroid 학습 세트 bag을 미리 할당한다.
train_centroids = np.zeros((df_train["review"].size, num_clusters),
                          dtype = "float32")

train_centroids[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [17]:
# centroid는 두 클러스터의 중심점을 정의 한 다음 중심점의 거리를 측정한 것
def create_bag_of_centroids(wordlist, word_centroid_map):
    
    # 클러스터의 수는 word/centroid map에서 가장 높은 클러스트 인덱스와 같다.
    num_centroids = max(word_centroid_map.values())+1
    
    # 속도를 위해 bag of centroids vector를 미리 할당한다.
    bag_of_centroids = np.zeros(num_centroids, dtype = "float32")
    
    # 루프를 돌며 단어가 word_centroid_map에 있다면
    # 해당되는 클러스터의 수를 하나씩 증가시켜 준다.
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
            
    # bag of centroids를 반환한다.
    return bag_of_centroids

In [18]:
# 학습 리뷰를 bags of centroids로 변환한다.
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids(review,
                                                      word_centroid_map)
    counter += 1

# 테스트 리뷰도 같은 방법으로 반복해 준다.
test_centroids = np.zeros((df_test["review"].size, num_clusters),
                         dtype = "float32")

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids(review,
                                                      word_centroid_map)
    counter += 1

In [19]:
# RandomForest를 사용하여 학습시키고 예측
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 100)

# train 데이터의 레이블을 통해 학습시키고 예측한다.
%time rf = rf.fit(train_centroids, df_train["sentiment"])

CPU times: user 52.4 s, sys: 51.4 ms, total: 52.4 s
Wall time: 52.4 s


In [20]:
from sklearn.model_selection import cross_val_score
%time score = np.mean(cross_val_score(rf, train_centroids,\
                                df_train["sentiment"],\
                                      cv = 10,\
                                      scoring = "roc_auc"))

CPU times: user 5min 51s, sys: 681 ms, total: 5min 52s
Wall time: 5min 52s


In [21]:
%time result = rf.predict(test_centroids)

CPU times: user 1.68 s, sys: 1.35 ms, total: 1.68 s
Wall time: 1.68 s


In [22]:
score

0.921640448

In [23]:
output = pd.DataFrame(data = {"id": df_test["id"], "sentiment":result})
output.to_csv("./submit_BagofCentroids_{:.3f}.csv".format(score),index = False,quoting = 3)

## 왜 이 튜토리얼에서는 BoW가 더 좋은 결과를 가져올까?

- 벡터를 평균화하고 centroids를 사용하면 단어 순서가 어긋나며 BoW 개념과 매우 비슷하다. 성능이(표준 오차의 범위 내에서)비슷하기 때문에 튜토리얼 1, 2, 3이 동등한 결과를 가져온다.

- 1) Word2Vec을 더 많은 텍스트로 학습시키면 성능이 좋아진다. Google의 결과는 10억 단어가 넘는 corpus에서 배운 단어 벡터를 기반으로 한다. 학습레이블이 있거나 레이블이 없는 학습 세트는 단지 대력 천팔백만 단어 정도다. 편의상 Word2Vec은 Google의 원래 C도구에서 출력되는 사전 학습된 모델을 로드하는 기능을 제공하기 때문에 C로 모델을 학습한 다음 Python으로 가져올 수도 있다.

- 2) 출판 된 자료들에서 분산 워드 벡터 기술은 BoW 모델보다 우수한 것으로 나타났다. 이 논문에서는 IMDb 데이터 집합에 단락 벡터(Paragraph Vector)라는 알고리즘을 사용하여 현재까지의 최첨단 결과 중 일부를 생성한다. 단락 벡터는 단어 순서 정보를 보존하는 반면 벡터 평균화 및 클러스터링은 단어 순서를 잃어 버리기 때문에 여기에서 시도하는 방식보다 부분적으로 좋다.